<a href="https://colab.research.google.com/github/89DAN89/89DAN89.github.io/blob/master/Saving_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [34]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [35]:
from google.colab import files
data_to_load = files.upload()

Saving dep.csv to dep (1).csv


## Data Pre-Processing

In [36]:
import io
dep = pd.read_csv(io.BytesIO(data_to_load['dep.csv']))

X = dep.drop("depressed", axis=1)
y = dep["depressed"]
print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

(1429, 22) (1429,)


# Create a Deep Learning Model

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=22))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [38]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

## Quantify our Trained Model

In [39]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: nan - accuracy: 0.8520
Normal Neural Network - Loss: nan, Accuracy: 0.8519552946090698


# Saving a Trained Model
We can save our trained models using the HDF5 binary format with the extension `.h5`

In [40]:
# Save the model
model.save("dep_model_trained.h5")

# Loading a Model

In [41]:
# Load the model
from tensorflow.keras.models import load_model
dep_model = load_model("dep_model_trained.h5")

## Evaluating the loaded model

In [42]:
model_loss, model_accuracy = dep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: nan - accuracy: 0.8520
Normal Neural Network - Loss: nan, Accuracy: 0.8519552946090698
